In [2]:


import tensorflow.compat.v1 as tf
import numpy as np
import random
from collections import defaultdict
from os import listdir
from os.path import isfile
import re
import keras 
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
tf.reset_default_graph()

def save_parameters(name,value,epoch):
  filename = name.replace(':','-colon-') + '-epoch-{}.txt'.format(epoch)
  if len(value.shape) == 1:
    string_form = ','.join([str(number) for number in value])
  else:
    string_form = '\n'.join([','.join([str(number) for number in value[row]]) for row in range(value.shape[0])])
  with open('D:/E/CUONG/machine learning/20news-bydate/saved-paras/' + filename,'w') as f:
      f.write(string_form)
def restore_parameters(name, epoch):
  filename = name.replace(':','-colon-') + '-epoch-{}.txt'.format(epoch)
  with open('D:/E/CUONG/machine learning/20news-bydate/saved-paras/' + filename) as f:
    lines = f.read().splitlines()
    if len(lines) == 1:
      value = [float(number) for number in lines[0].split(',')]
    else:
      value = [[float(number) for number in lines[row].split(',')] for row in range(len(lines))]
  return value
class MLP:
  def __init__(self, vocab_size, hidden_size):
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
  def build_graph(self):
    NUM_CLASSES = 100
    self._X = tf.placeholder(tf.float32, shape = [None, self._vocab_size])
    self._real_Y = tf.placeholder(tf.int32, shape = [None, ])

    weights_1 = tf.get_variable(name = 'weights_input_hidden', shape = (self._vocab_size, self._hidden_size), initializer = tf.random_normal_initializer(seed = 2018))
    biases_1 = tf.get_variable(name = 'biases_input_hidden', shape = (self._hidden_size), initializer = tf.random_normal_initializer(seed = 2018))
    weights_2 = tf.get_variable(name = 'weights_hidden_output', shape = (self._hidden_size, NUM_CLASSES), initializer = tf.random_normal_initializer(seed = 2018))
    biases_2 = tf.get_variable(name = 'biases_hidden_output', shape = (NUM_CLASSES), initializer = tf.random_normal_initializer(seed = 2018))
    hidden = tf.matmul(self._X, weights_1) + biases_1
    hidden = tf.sigmoid(hidden)
    logits = tf.matmul(hidden, weights_2) + biases_2
    labels_one_hot = tf.one_hot(indices=self._real_Y, depth=NUM_CLASSES, dtype=tf.float32)
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits=logits)
    loss = tf.reduce_mean(loss)
    probs = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(probs, axis = 1)
    predicted_labels = tf.squeeze(predicted_labels)
    return predicted_labels, loss
  def trainer(self, loss, learning_rate):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_op
with open('D:/E/CUONG/machine learning/20news-bydate/words_idfs.txt') as f:
    vocab_size = len(f.read().splitlines())
    mlp = MLP(vocab_size = vocab_size, hidden_size = 50)
    predicted_labels, loss = mlp.build_graph()
    train_op = mlp.trainer(loss = loss, learning_rate = 0.1)


class DataReader:
  def __init__(self, data_path, batch_size, vocab_size):
    self._batch_size = batch_size
    with open(data_path) as f:
      d_lines = f.read().splitlines()
    self._data = []
    self._labels =[]
    for data_id, line in enumerate(d_lines):
      vector = [0.0 for _ in range(vocab_size)]
      features = line.split('<fff>')
      label, doc_id = int(features[0]), int(features[1])
      tokens = features[2].split()
      for token in tokens:
        index, value = int(token.split(':')[0]), float(token.split(':')[1])
        vector[index] = value
      self._data.append(vector)
      self._labels.append(label)
    self._data = np.array(self._data)
    self._labels = np.array(self._labels)
    self._num_epoch = 0
    self._batch_id = 0
  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1
    if end + self._batch_size > len(self._data):
      end = len(self._data)
      self._num_epoch += 1
      self._batch_id = 0
      indices = list(range(len(self._data)))
      random.seed(2018)
      random.shuffle(indices)
      self._data, self._labels = self._data[indices], self._labels[indices]
    return self._data[start:end], self._labels[start:end]
def load_dataset():
    train_data_reader = DataReader(data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-train-tfidf.txt',batch_size = 50, vocab_size = vocab_size)
    test_data_reader = DataReader(data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-test-tfidf.txt',batch_size = 50, vocab_size = vocab_size)
    return train_data_reader, test_data_reader    

  
with tf.Session() as sess:
    
    train_data_reader, test_data_reader = load_dataset()
    step, MAX_STEP = 0, 36000
    sess.run(tf.global_variables_initializer())
    while step < MAX_STEP:
      train_data, train_labels = train_data_reader.next_batch()
      plabels_eval, loss_eval, _ = sess.run([predicted_labels, loss, train_op], feed_dict={mlp._X: train_data, mlp._real_Y: train_labels})
      


      step += 1
      print('step: {}, loss: {}'.format(step, loss_eval))
    trainable_variables = tf.trainable_variables()
    
    for variable in trainable_variables:
        #print(variable.name)
        #print(variable.eval())
        save_parameters(name = variable.name, value = variable.eval(), epoch = train_data_reader._num_epoch)
    test_data_reader = DataReader(data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-test-tfidf.txt',batch_size = 50, vocab_size = vocab_size)

    with tf.Session() as sess:
        epoch = 0
        for variable in trainable_variables:
            saved_value = restore_parameters(variable.name, epoch)
            assign_op = variable.assign(saved_value)
            sess.run(assign_op)
        num_true_preds = 0
        while True:
            test_data, test_labels = test_data_reader.next_batch()
            test_plabels_eval = sess.run(predicted_labels, feed_dict = {mlp._X: test_data, mlp._real_Y:test_labels})
            matches = np.equal(test_plabels_eval, test_labels)
            num_true_preds += np.sum(matches.astype(float))
            if test_data_reader._batch_id == 0:
                break
        print('Epoch:',test_data_reader._num_epoch)
        print('Accuracy on test data:', num_true_preds / len(test_data_reader._data))

        








step: 1, loss: 4.169332027435303
step: 2, loss: 0.003972718492150307
step: 3, loss: 2.6313202397432178e-05
step: 4, loss: 5.36440438736463e-07
step: 5, loss: 3.0994396382766354e-08
step: 6, loss: 2.38418573772492e-09
step: 7, loss: 0.0
step: 8, loss: 0.0
step: 9, loss: 0.0
step: 10, loss: 14.665328025817871
step: 11, loss: 32.89753341674805
step: 12, loss: 29.00861167907715
step: 13, loss: 23.246826171875
step: 14, loss: 16.935144424438477
step: 15, loss: 10.153888702392578
step: 16, loss: 3.7774713039398193
step: 17, loss: 0.38595789670944214
step: 18, loss: 0.006771399173885584
step: 19, loss: 2.5346902475575916e-05
step: 20, loss: 2.431865482321882e-07
step: 21, loss: 3.8146950487316644e-08
step: 22, loss: 17.639118194580078
step: 23, loss: 23.644428253173828
step: 24, loss: 21.5782470703125
step: 25, loss: 18.51752471923828
step: 26, loss: 15.378205299377441
step: 27, loss: 11.579537391662598
step: 28, loss: 7.943426609039307
step: 29, loss: 3.9362692832946777
step: 30, loss: 1.240

In [11]:
def gen_data_and_vocab():
    def collect_data_from(parent_path, newsgroup_list, word_count = None):
        data = []
        for group_id, newsgroup in enumerate(newsgroup_list):
            dir_path = parent_path + '/' + newsgroup +'/'
            files = [(filename, dir_path + filename) for filename in listdir(dir_path) if isfile(dir_path+filename)]
            files.sort()
            label = group_id
            print('Processing: {}-{}'.format(group_id, newsgroup))
            for filename, filepath in files:
                with open(filepath) as f:
                    text = f.read().lower()
                    words = re.split('\W+', text)
                    if word_count is not None:
                        for word in words:
                            word_count[word] +=1
                    content = ' '.join(words)
                    assert len(content.splitlines()) == 1
                    data.append(str(label)+ '<fff>' + filename +'<fff>' + content)
        return data
    word_count = defaultdict(int)
    path = 'D:/E/CUONG/machine learning/20news-bydate/'
    parts = [path+ dir_name + '/' for dir_name in listdir(path) if not isfile(path + dir_name)]
    train_path, test_path = (parts[0], parts[1]) if 'train' in parts[0] else (parts[1], parts[0])
    newsgroup_list = [newsgroup for newsgroup in listdir(train_path)]
    newsgroup_list.sort()
    train_data = collect_data_from(parent_path = train_path, newsgroup_list = newsgroup_list, word_count = word_count)
    vocab = [word for word,freq in zip(word_count.keys(), word_count.values()) if freq >10]
    vocab.sort()
    with open('D:/E/CUONG/machine learning/w2v/vocab-raw.txt', 'w') as f:
        f.write('\n'.join(vocab))
    test_data = collect_data_from(parent_path = test_path, newsgroup_list = newsgroup_list)
    with open('D:/E/CUONG/machine learning/w2v/20news-train-raw.txt', 'w') as f:
        f.write('\n'.join(train_data))
    with open('D:/E/CUONG/machine learning/w2v/20news-test-raw.txt', 'w') as f:
        f.write('\n'.join(test_data))

    
gen_data_and_vocab()                         
                                                                                                 

Processing: 0-alt.atheism
Processing: 1-comp.graphics
Processing: 2-comp.os.ms-windows.misc
Processing: 3-comp.sys.ibm.pc.hardware
Processing: 4-comp.sys.mac.hardware
Processing: 5-comp.windows.x
Processing: 6-misc.forsale
Processing: 7-rec.autos
Processing: 8-rec.motorcycles
Processing: 9-rec.sport.baseball
Processing: 10-rec.sport.hockey
Processing: 11-sci.crypt
Processing: 12-sci.electronics
Processing: 13-sci.med
Processing: 14-sci.space
Processing: 15-soc.religion.christian
Processing: 16-talk.politics.guns
Processing: 17-talk.politics.mideast
Processing: 18-talk.politics.misc
Processing: 19-talk.religion.misc
Processing: 0-alt.atheism
Processing: 1-comp.graphics
Processing: 2-comp.os.ms-windows.misc
Processing: 3-comp.sys.ibm.pc.hardware
Processing: 4-comp.sys.mac.hardware
Processing: 5-comp.windows.x
Processing: 6-misc.forsale
Processing: 7-rec.autos
Processing: 8-rec.motorcycles
Processing: 9-rec.sport.baseball
Processing: 10-rec.sport.hockey
Processing: 11-sci.crypt
Processing

In [30]:
def encode_data(data_path, vocab_path):
    with open(vocab_path) as f:
        vocab = dict([(word, word_ID + 2) for word_ID, word in enumerate(f.read().splitlines())])
    with open(data_path) as f:
        documents = [(line.split('<fff>')[0], line.split('<fff>')[1], line.split('<fff>')[2]) for line in f.read().splitlines()]
    encoded_data = []
    for document in documents:
        label, doc_id, text = document
        words = text.split()[:500]
        sentence_length = len(words)
        encoded_text =[]
        for word in words:
            if word in vocab:
                encoded_text.append(str(vocab[word]))
            else:
                encoded_text.append(str(1))
        if len(words) < 500:
            num_padding = 500 - len(words)
            for _ in range(num_padding):
                encoded_text.append(str(0))
        encoded_data.append(str(label) + '<fff>' + str(doc_id) +'<fff>' + str(sentence_length)+'<fff>'+ ' '.join(encoded_text))
    dir_name = '/'.join(data_path.split('/')[:-1])
    file_name ='-'.join(data_path.split('/')[-1].split('-')[:-1]) + '-encoded.txt'
    with open(dir_name + '/' + file_name, 'w') as f:
        f.write('\n'.join(encoded_data))
data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-test-processed.txt'
vocab_path = 'D:/E/CUONG/machine learning/w2v/vocab-raw.txt'
encode_data(data_path,vocab_path)
        

In [4]:
tf.reset_default_graph()
class DataReader_session_4:
  def __init__(self, data_path, batch_size, vocab_size):
    self._batch_size = batch_size
    with open(data_path) as f:
      d_lines = f.read().splitlines()
    self._data = []
    self._labels =[]
    self._sentence_lengths = []
    self._final_tokens = []
    for data_id, line in enumerate(d_lines):
      vector = []
      features = line.split('<fff>')
      label, doc_id, sentence_lengths = int(features[0]), int(features[1]), int(features[2])
      tokens = features[3].split()
      for token in tokens:
        vector.append(token)
      self._data.append(vector)
      self._labels.append(label)
      self._sentence_lengths.append(sentence_lengths)
      self._final_tokens.append(tokens[-1])
    self._data = np.array(self._data)
    self._labels = np.array(self._labels)
    self._sentence_lengths = np.array(self._sentence_lengths)
    self._final_tokens = np.array(self._final_tokens)
    self._size = len(self._data)
    self._num_epoch = 0
    self._batch_id = 0
  def next_batch(self):
    start = self._batch_id * self._batch_size
    end = start + self._batch_size
    self._batch_id += 1

    if end + self._batch_size > len(self._data):
      self._size = end
      end = len(self._data)
      start = end - self._batch_size
      self._num_epoch += 1
      self._batch_id = 0
      indices = list(range(len(self._data)))
      random.seed(2021)
      random.shuffle(indices)
      self._data, self._labels, self._sentence_lengths, self._final_tokens = self._data[indices], self._labels[indices], self._sentence_lengths[indices], self._final_tokens[indices]

    return self._data[start:end], self._labels[start:end], self._sentence_lengths[start:end], self._final_tokens[start:end]
class RNN:
    def __init__(self,vocab_size, embedding_size, lstm_size,batch_size):
        self._vocab_size = vocab_size
        self._embedding_size = embedding_size
        self._lstm_size = lstm_size
        self._batch_size = batch_size
        
        self._data = tf.placeholder(tf.int32, shape = [batch_size,500])
        self._labels = tf.placeholder(tf.int32, shape = [batch_size, ])
        self._sentence_lengths = tf.placeholder(tf.int32,shape = [batch_size, ])
        self._final_tokens = tf.placeholder(tf.int32, shape = [batch_size, ])
    def embedding_layer(self,indices):
        pretrained_vectors =[]
        pretrained_vectors.append(np.zeros(self._embedding_size))
        np.random.seed(2018)
        for _ in range(self._vocab_size + 1):
            pretrained_vectors.append(np.random.normal(loc = 0., scale = 1., size = self._embedding_size))
        pretrained_vectors = np.array(pretrained_vectors)
        self._embedding_matrix = tf.get_variable(name = 'embedding', shape = (self._vocab_size + 2, self._embedding_size),initializer = tf.constant_initializer(pretrained_vectors))
        return tf.nn.embedding_lookup(self._embedding_matrix, indices)
    def LSTM_layer(self, embeddings):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(self._lstm_size)
        zero_state = tf.zeros(shape = (self._batch_size, self._lstm_size))
        initial_state = tf.nn.rnn_cell.LSTMStateTuple(zero_state, zero_state)
        lstm_inputs = tf.unstack(tf.transpose(embeddings, perm = [1,0,2]))
        lstm_outputs, last_state = tf.nn.static_rnn(cell = lstm_cell, inputs = lstm_inputs, initial_state = initial_state, sequence_length = self._sentence_lengths)
        lstm_outputs = tf.unstack(tf.transpose(lstm_outputs, perm = [1,0,2]))
        lstm_outputs = tf.concat(lstm_outputs, axis =0)
        mask = tf.sequence_mask(lengths = self._sentence_lengths, maxlen = 500, dtype = tf.float32)
        mask = tf.concat(tf.unstack(mask, axis =0), axis =0)
        mask = tf.expand_dims(mask, -1)
        lstm_outputs = mask * lstm_outputs
        lstm_outputs_split = tf.split(lstm_outputs, num_or_size_splits = self._batch_size)
        lstm_outputs_sum = tf.reduce_sum(lstm_outputs_split,axis = 1)
        lstm_outputs_average = lstm_outputs_sum / tf.expand_dims(tf.cast(self._sentence_lengths, tf.float32),-1)
        return lstm_outputs_average
    def build_graph(self):
        NUM_CLASSES = 20
        embeddings = self.embedding_layer(self._data)
        lstm_outputs = self.LSTM_layer(embeddings)
        
        weights = tf.get_variable(name = 'final_layer_weights', shape =(self._lstm_size, NUM_CLASSES), initializer = tf.random_normal_initializer(seed = 2018))
        biases = tf.get_variable(name = 'final_layer_biases', shape =(NUM_CLASSES), initializer = tf.random_normal_initializer(seed = 2018))
        logits = tf.matmul(lstm_outputs, weights) + biases
        logits = tf.matmul(lstm_outputs, weights) + biases
        labels_one_hot = tf.one_hot(indices = self._labels, depth = NUM_CLASSES, dtype = tf.float32)
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot, logits=logits)
        loss = tf.reduce_mean(loss)
        probs = tf.nn.softmax(logits)
        predicted_labels = tf.argmax(probs, axis = 1)
        predicted_labels = tf.squeeze(predicted_labels)
        return predicted_labels, loss
    def trainer(self, loss, learning_rate):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        return train_op
def train_and_evaluate_RNN():
        with open('D:/E/CUONG/machine learning/w2v/vocab-raw.txt') as f:
            vocab_size = len(f.read().splitlines())
        tf.set_random_seed(2018)
        rnn = RNN(vocab_size = vocab_size, embedding_size = 300, lstm_size = 50, batch_size = 50)
        predicted_labels, loss = rnn.build_graph()
        train_op = rnn.trainer(loss = loss , learning_rate = 0.01)
        with tf.Session() as sess:
            train_data_reader = DataReader_session_4(data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-train-encoded.txt', batch_size = 50,vocab_size = vocab_size)
            test_data_reader = DataReader_session_4(data_path = 'D:/E/CUONG/machine learning/20news-bydate/20news-test-encoded.txt', batch_size = 50, vocab_size = vocab_size)
            #print(len(train_data_reader._data),len(train_data_reader._data[0]))
            step = 0
            MAX_STEP = 1000
            sess.run(tf.global_variables_initializer())
            while step < MAX_STEP:
                next_train_batch = train_data_reader.next_batch()
                train_data, train_labels, train_sentence_lengths, train_final_tokens = next_train_batch
                print(len(train_data),len(train_data[0]))
                plabels_eval, loss_eval, _ = sess.run([predicted_labels, loss, train_op], feed_dict = {rnn._data: train_data, rnn._labels: train_labels, rnn._sentence_lengths: train_sentence_lengths, rnn._final_tokens: train_final_tokens})
                step +=1
                if step % 20 ==0:
                    print('loss', loss_eval)
                if train_data_reader._batch_id ==0:
                    num_true_preds = 0
                    while True:
                        next_test_batch = test_data_reader.next_batch()
                        test_data, test_labels, test_sentence_lengths, test_final_tokens = next_test_batch
                        test_plabels_eval = sess.run(predicted_labels, feed_dict = {rnn._data: test_data, rnn._labels: test_labels, rnn._sentence_lengths: test_sentence_lengths, rnn._final_tokens: test_final_tokens})
                        matches = np.equal(test_plabels_eval, test_labels)
                        num_true_preds += np.sum(matches.astype(float))
                        if test_data_reader._batch_id == 0:
                            break
                    print('Epoch:', train_data_reader._num_epoch)
                    print('Accuracy on test data:', num_true_preds * 100./len(test_data_reader._data))
train_and_evaluate_RNN()            

                                           

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\Admin\anaconda3\envs\tf\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:901: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\Admin\anaconda3\envs\tf\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


11314 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 0.0076287673
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 0.93945414
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 7.727229
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 1.4979831
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 1.9450568
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
loss 7.4276824
50 500
50 500
50 500
50 500
50 500
50 500
50 500
50 500
5